In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
df = pd.read_csv('csv/movies.csv')

In [4]:
df.head()

,Unnamed: 0,id,keywords,overview,popularity,release_date,runtime,status,tagline,title,...,Denmark,Singapore,Romania,Malaysia,United States of America,Norway,Chile,Australia,United Arab Emirates,Iran
0,0,19995,culture clash;future;space war;space colony;so...,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009.0,162.0,Released,Enter the World of Pandora.,Avatar,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,285,ocean;drug abuse;exotic island;east india trad...,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007.0,169.0,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,206647,spy;based on novel;secret agent;sequel;mi6;bri...,A cryptic message from Bond’s past sends him o...,107.376788,2015.0,148.0,Released,A Plan No One Escapes,Spectre,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,49026,dc comics;crime fighter;terrorist;secret ident...,Following the death of District Attorney Harve...,112.312950,2012.0,165.0,Released,The Legend Ends,The Dark Knight Rises,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4,49529,based on novel;mars;medallion;space travel;pri...,"John Carter is a war-weary, former military ca...",43.926995,2012.0,132.0,Released,"Lost in our world, found in another.",John Carter,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [6]:
keywords = df['keywords'].dropna()

In [11]:
df = df.dropna(subset=['keywords'])

In [13]:
df.shape

(4386, 86)

In [16]:
titles = df['title']

In [18]:
np.save('titles.npy', np.array(titles))

In [30]:
# Для размышлений
from difflib import SequenceMatcher

space = tuple(filter(lambda x: x.lower().startswith('space'), keywords))

In [33]:
[(SequenceMatcher(None, space[0], i).ratio(), i) for i in space]

[(1.0, 'space marine'),
 (0.5833333333333334, 'space colony'),
 (0.6086956521739131, 'space opera'),
 (0.5882352941176471, 'space'),
 (0.5714285714285714, 'spaceship'),
 (0.5714285714285714, 'spaceport'),
 (0.7272727272727273, 'space race'),
 (0.72, 'space mission'),
 (0.6666666666666666, 'space pirate'),
 (0.8, 'space carrier'),
 (0.5333333333333333, 'space based weapon'),
 (0.72, 'space station'),
 (0.8, 'spaceman'),
 (0.6153846153846154, 'space invasion'),
 (0.5454545454545454, 'spacecraft'),
 (0.7619047619047619, 'space war'),
 (0.6363636363636364, 'space suit'),
 (0.6666666666666666, 'space travel'),
 (0.6666666666666666, 'space battle'),
 (0.5806451612903226, 'spacecraft accident'),
 (0.6666666666666666, 'space centre'),
 (0.4666666666666667, 'spacecraft officer')]

In [37]:
tokenizer = Tokenizer(num_words=8000,
                      lower=True,
                      split=';')

In [39]:
tokenizer.fit_on_texts(keywords)

In [65]:
len(keywords[0].split(';'))

21

In [ ]:
def to_binary(words_indexes, num_words=8000):
     binary_words = np.zeros(num_words)
     binary_words[words_indexes] = 1
     return binary_words

In [64]:
words = list(filter(bool, [x[0] if x else 0 for x in tokenizer.texts_to_sequences(keywords[0].split(';'))]))
print(len(words))
binary_words = to_binary(words)
binary_words

21


array([0., 0., 0., ..., 0., 0., 0.])

In [66]:
(binary_words == 1).sum(), (binary_words == 0).sum()

(21, 7979)

In [67]:
import pickle

with open('tokenizer.pickle', 'wb') as f:
    pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
from tokenizer import CustomTokenizer
t = CustomTokenizer()
all_x = []
for n, keyword in enumerate(keywords):
    all_x.append(np.array(t.get_binary_indexes(t.cut_words(keyword))))
    if n % 100 == 0:
        print('=', end='')

In [8]:
np.save('all_x.npy', np.array(all_x))

In [25]:
genres = df[['Western', 'Foreign', 'TV Movie', 'Comedy', 'Romance', 'Thriller', 'Documentary', 'Fantasy', 'Family', 'History', 'Crime', 'Horror', 'Mystery', 'Drama', 'Music', 'Science Fiction', 'Adventure', 'Animation', 'War', 'Action']]

In [27]:
np.save('genres.npy', np.array(genres))

In [30]:
titles = np.load('titles.npy', allow_pickle=True)
keywords = np.load('vecotors_decoded.npy', allow_pickle=True)
genres = np.load('genres.npy', allow_pickle=True)

In [31]:
titles.shape, keywords.shape, genres.shape

((4386,), (4386, 8000), (4386, 20))

In [40]:
keywords_with_genres = np.concatenate((keywords, genres), axis=1)

In [41]:
np.save('keywords_with_genres.npy', keywords_with_genres)